## Домашнее задание №3 ##
<ol>
<li>обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
<li>при обучении моделей обязательно использовать кроссвалидацию
<li>вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
<li>сделать выводы о том, какая модель справилась с задачей лучше других
<li>(опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).
</ol>

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from my_unit import ColumnSelector, NumberSelector, OHEEncoder

import warnings
warnings.filterwarnings('ignore')

**1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта**

In [2]:
df = pd.read_csv("train_case2.csv",delimiter=';')
print(df.shape)
df.head(3)

(70000, 13)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [4]:
X_train.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
4497,6369,15212,2,170,85.0,130,80,3,3,1,0,1
8301,11845,15361,1,165,70.0,110,70,1,1,0,0,1
63679,90920,19822,1,174,72.0,120,80,1,1,0,0,0


In [5]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']


continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [7]:
def scores(classifier):
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score
    # Исползуем Бета  = 1
    b = 1
    # Воспользуемся кроссвалидацией
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    # расчитаем метрики
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc_score = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score]]

Запустим для четырех 'LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier', 'XGBClassifier' моделей.

In [8]:
%%time 

results = pd.DataFrame([
    scores(classifier=Pipeline([('features', feats), ('classifier', LogisticRegression(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth=4, random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', XGBClassifier(max_depth=4, eval_metric='mlogloss', random_state=42))])),
                        ], columns=['CV_score', 'BestThreshold', 'FScore', 'Precision', 'Recall', 'ROCAUC'])

Wall time: 2min 3s


In [9]:
results['models_name'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier', 'XGBClassifier' ]
results

,CV_score,BestThreshold,FScore,Precision,Recall,ROCAUC,models_name
0,0.787,0.387,0.730,0.647,0.838,0.784,LogisticRegression
1,0.773,0.350,0.719,0.643,0.816,0.771,RandomForestClassifier
2,0.787,0.380,0.731,0.647,0.840,0.789,DecisionTreeClassifier
3,0.801,0.371,0.738,0.679,0.810,0.801,XGBClassifier


**4. Вывод**

Если судить по ROC_AUC, то наилучшим из представленных моделей является XGBClassifier, но в данной задаче нам лучше использовать Recall чем Precision, стремясь увеличиват TP и уменьшая FN результаты. В нашем случае нам больше подойдет дерево решений DecisionTreeClassifier.


**5. precision и recall не зависят от соотношения классов и более применимы в условиях не сбалансируемых выборок, чем accuracy**